Cross Validation 2

size =  121655

test size 4k+

epoch-5

apply sigmoid (prediction probabilities are logits)

groups-9

target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

threshold-tuning = yes

df_2k = df[(df['Rating'] >= 4.0) & (df['Maximum_Installs'] >= 5000)]

accuracy score : F1 score (micro) and ROC_AUC score

## 1. Setup

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
print(tf.__version__)

2.8.0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load Dataset

In [4]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [5]:
## uploading csv files on drive (to avoid uploading on colab in every session)

from google.colab import drive
drive.mount("/content/drive/")

## drive path
train_path = "/content/drive/MyDrive/MetadataCSV/data_121655/CV_df_train_2.csv"
val_path = "/content/drive/MyDrive/MetadataCSV/data_121655/CV_df_val_2.csv"
test_path = "/content/drive/MyDrive/MetadataCSV/test_dataset.csv"

Mounted at /content/drive/


In [6]:
df_train = pd.read_csv(train_path) 
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(97324, 23)
(24331, 23)
(4624, 23)


In [7]:
df_train.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Science Kids Learning - Be Super Scientist!,com.sciencegames.ScienceKidsLearning,Educational,4.2,78871,False,Science is all about inspiring curiosity and d...,https://nutboltgames.com/privacy-policy,0,0,...,0,0,1,0,0,0,1846,science is all about inspiring curiosity and d...,1807,343
1,Phuti kismat se Bhagyashali,com.photi.bhgysl,Lifestyle,4.4,13681,False,This year has in its fold mixed fate to offer ...,https://labtest12.wixsite.com/privacypolicy,0,0,...,0,0,0,0,0,0,1513,this year has in its fold mixed fate to offer ...,1443,350


In [8]:
target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

In [9]:
# getting number of nonzeros in each column
df_train[target_list].astype(bool).sum(axis=0)

Camera        16337
Location      16300
Microphone     8821
Contacts      10476
Storage       48246
Phone         15079
SMS             311
Call_Log        201
Calendar       1775
dtype: int64

In [10]:
df_val[target_list].astype(bool).sum(axis=0)

Camera         4116
Location       4042
Microphone     2220
Contacts       2644
Storage       12012
Phone          3798
SMS             102
Call_Log         65
Calendar        466
dtype: int64

In [11]:
df_test[target_list].astype(bool).sum(axis=0)

Camera         745
Location       716
Microphone     436
Contacts       500
Storage       2402
Phone          652
SMS             11
Call_Log         6
Calendar        90
dtype: int64

## 3. Data Preprocess

#### 3.1 Clean Text

In [12]:
import nltk

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [14]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(remove_stopwords)
df_val["Clean_Description"] = df_val["Clean_Description"].map(remove_stopwords)
df_test["Clean_Description"] = df_test["Clean_Description"].map(remove_stopwords)

In [15]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [16]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(lambda x: remove_punct(x))
df_val["Clean_Description"] = df_val["Clean_Description"].map(lambda x: remove_punct(x))
df_test["Clean_Description"] = df_test["Clean_Description"].map(lambda x: remove_punct(x))

#### 3.2 Create Corpus

In [17]:
from keras.layers import *

In [18]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [19]:
from nltk.tokenize import word_tokenize
import nltk


def create_corpus_tk(df):
    corpus = []
    for text in df["Clean_Description"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [20]:
df_T = df_train.append(df_val)
df_T.reset_index(drop=True,inplace=True)

In [21]:
corpus = create_corpus_tk(df_T)

In [22]:
num_words = len(corpus)
print(num_words)

121655


In [23]:
corpus[0]

['science',
 'inspiring',
 'curiosity',
 'discovering',
 'world',
 'learn',
 'science',
 'fun',
 'educational',
 'games',
 'science',
 'games',
 'sure',
 'fulfill',
 'sense',
 'curiosity',
 'introduce',
 'basic',
 'science',
 'concepts',
 'approachable',
 'manner',
 'reinforcing',
 'comprehension',
 'engaging',
 'stimulating',
 'activities',
 'using',
 'interactive',
 'science',
 'games',
 'creates',
 'unique',
 'learning',
 'process',
 'combined',
 'conventional',
 'topics',
 'food',
 'chain',
 'learn',
 'food',
 'chain',
 'food',
 'web',
 'energy',
 'cycles',
 'transfers',
 'living',
 'organisms',
 'lesson',
 'learn',
 'simple',
 'food',
 'chains',
 'energy',
 'passed',
 'producers',
 'consumers',
 'decomposers',
 'eats',
 'what',
 'energy',
 'life',
 'needs',
 'survive',
 'come',
 'from',
 'learn',
 'producers',
 'primary',
 'consumers',
 'secondary',
 'tertiary',
 'consumers',
 'apex',
 'predators',
 'decomposers',
 'roles',
 'play',
 'food',
 'chains',
 'microorganisms',
 'ever',


##4. Train Validation Split

In [ ]:
# split the data into a training set and a validation set

In [24]:
train_inputs = df_train["Clean_Description"]
validation_inputs = df_val["Clean_Description"]
train_labels = df_train[target_list]
validation_labels = df_val[target_list]

## 5. Tokenization

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_inputs)

In [26]:
MAX_SEQUENCE_LENGTH = 600

In [27]:
train_sequences = tokenizer.texts_to_sequences(train_inputs)

In [28]:
train_inputs

0        science inspiring curiosity discovering world ...
1        year fold mixed fate offer fellow human beings...
2        realpi provides best interesting pi calculatio...
3        regularly helps keep track repeating tasks fix...
4        harri live highly intuitive useful team schedu...
                               ...                        
97319    stylist wallpapers keyboard beautiful keyboard...
97320    consists precise easy understand meanings phys...
97321    may light guide dark places pendulum artifact ...
97322    leo browser made minimalism comfortable web br...
97323    boltnet vpn protects internet session helps ac...
Name: Clean_Description, Length: 97324, dtype: object

In [29]:
train_sequences

[[567,
  3038,
  6123,
  6461,
  94,
  31,
  567,
  96,
  460,
  71,
  567,
  71,
  420,
  4992,
  2190,
  6123,
  3336,
  387,
  567,
  1402,
  27207,
  1881,
  19332,
  3670,
  2361,
  8500,
  616,
  38,
  619,
  567,
  71,
  2865,
  188,
  73,
  523,
  3005,
  6049,
  370,
  421,
  3398,
  31,
  421,
  3398,
  421,
  346,
  983,
  4506,
  1756,
  1676,
  8029,
  1724,
  31,
  47,
  421,
  6348,
  983,
  4839,
  8416,
  4324,
  70778,
  13377,
  5732,
  983,
  87,
  424,
  6093,
  510,
  1378,
  31,
  8416,
  2358,
  4324,
  3183,
  20263,
  4324,
  1783,
  14007,
  70778,
  6503,
  68,
  421,
  6348,
  20738,
  494,
  6528,
  3204,
  141,
  13524,
  31895,
  31895,
  31895,
  18510,
  321,
  1837,
  2120,
  156,
  31,
  383,
  1676,
  3913,
  8029,
  17,
  11363,
  5375,
  13378,
  24089,
  36724,
  1200,
  348,
  295,
  2243,
  31,
  1676,
  527,
  1418,
  418,
  1005,
  3,
  429,
  421,
  674,
  180,
  1200,
  41,
  418,
  1005,
  382,
  8985,
  545,
  145,
  94,
  567,
  31,
  53

In [30]:
train_padded = pad_sequences(
    train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating="post", padding="post"
)

In [31]:
train_padded

array([[  567,  3038,  6123, ...,     0,     0,     0],
       [  268,  6882,  3267, ...,     0,     0,     0],
       [40107,   148,    15, ...,     0,     0,     0],
       ...,
       [  168,   292,   482, ...,     0,     0,     0],
       [ 4516,   583,   259, ...,     0,     0,     0],
       [  214,  3262,    83, ...,     0,     0,     0]], dtype=int32)

In [32]:
validation_sequences = tokenizer.texts_to_sequences(validation_inputs)
validation_padded = pad_sequences(
    validation_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [33]:
validation_padded

array([[   1,  375,  360, ...,    0,    0,    0],
       [  58,    1, 1686, ...,    0,    0,    0],
       [4518, 8141,  174, ...,    0,    0,    0],
       ...,
       [ 416,   66,  458, ...,    0,    0,    0],
       [1360,  378, 1815, ...,    0,    0,    0],
       [  98,    1, 1720, ...,    0,    0,    0]], dtype=int32)

In [34]:
print(df_train.Clean_Description[0])
print(train_sequences[0])

science inspiring curiosity discovering world learn science fun educational games science games sure fulfill sense curiosity introduce basic science concepts approachable manner reinforcing comprehension engaging stimulating activities using interactive science games creates unique learning process combined conventional topics food chain learn food chain food web energy cycles transfers living organisms lesson learn simple food chains energy passed producers consumers decomposers eats what energy life needs survive come from learn producers primary consumers secondary tertiary consumers apex predators decomposers roles play food chains microorganisms ever wondered happens look microscope trillions trillions trillions microbes around earth maybe more learn types living micro organisms like bacteria viruses fungi algae protozoa human body health growth learn living things grow stay healthy use water food exercise provide human need stay healthy super scientist enter amazing world science

In [35]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 195246


In [36]:
word_index

{'app': 1,
 'free': 2,
 'use': 3,
 'keyboard': 4,
 'wallpaper': 5,
 'theme': 6,
 'phone': 7,
 'application': 8,
 'features': 9,
 'new': 10,
 'get': 11,
 'wallpapers': 12,
 'time': 13,
 'also': 14,
 'best': 15,
 'screen': 16,
 'like': 17,
 'one': 18,
 'make': 19,
 'download': 20,
 'easy': 21,
 'android': 22,
 'launcher': 23,
 'us': 24,
 'english': 25,
 'live': 26,
 'mobile': 27,
 'find': 28,
 'share': 29,
 'help': 30,
 'learn': 31,
 'hd': 32,
 'love': 33,
 'set': 34,
 'device': 35,
 'video': 36,
 'please': 37,
 'using': 38,
 'apps': 39,
 'want': 40,
 'need': 41,
 'images': 42,
 'information': 43,
 'support': 44,
 'home': 45,
 'themes': 46,
 'simple': 47,
 'friends': 48,
 'many': 49,
 'text': 50,
 'enjoy': 51,
 'available': 52,
 'different': 53,
 'data': 54,
 'language': 55,
 'access': 56,
 'day': 57,
 'beautiful': 58,
 'game': 59,
 'easily': 60,
 'add': 61,
 'user': 62,
 'online': 63,
 'choose': 64,
 'you': 65,
 'way': 66,
 'words': 67,
 'play': 68,
 'create': 69,
 'every': 70,
 'games'

In [37]:
word_index["reason"]

2385

In [38]:
print(validation_sequences[0])

[1, 375, 360, 41, 291, 45, 47, 1697, 226, 375, 318, 982, 658, 21, 177, 45, 2833, 982, 658, 154, 1639, 665, 226, 7211, 16, 1972, 93, 43379, 147, 3010, 780, 45, 16, 375, 279, 364, 364, 364, 14, 93, 279, 2322, 34, 45, 147, 3, 982, 658, 100, 982, 658, 182, 1593, 235, 93, 82, 50, 279, 182, 1593, 1282, 2416, 93, 658, 6827, 182, 261, 3, 43379, 81, 93, 3304, 701, 100, 6, 1025, 109, 592, 290, 125, 1432, 9711, 656, 1197, 79, 2656, 7, 453, 30174, 265, 3785, 39, 37, 7009, 1, 703, 79, 4364, 793, 800, 350, 89, 690, 22, 793, 101, 1, 3310, 7, 64, 112, 412, 579, 2029, 213, 838, 579, 322, 112, 4140, 1, 40, 93, 147, 213, 1, 125, 112, 4140, 212, 571, 2458, 793, 134, 2458, 5709, 680, 982, 658, 1233, 5265, 17, 1, 4125, 154, 1242, 3055, 181, 154]


In [39]:
word_index["listen"]

388

## 6. Create the Embedding dictionary

In [40]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove6B/glove.6B.300d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [41]:
embedding_dict

Output hidden; open in https://colab.research.google.com to view.

In [42]:
embedding_dim = 300

In [43]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [44]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.73247999, -0.078309  ,  0.018521  , ...,  0.21988   ,
        -0.13121   ,  0.043819  ],
       [-0.30414   , -0.37029999, -0.15881   , ..., -0.37830999,
        -0.42910001,  0.0030023 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.62230998, -0.40289   ,  0.11949   , ...,  0.93154001,
         0.22446001,  0.15016   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [45]:
word_index["reason"]

2385

In [46]:
embedding_dict.get("reason")

array([ 1.3197e-01, -1.2591e-01,  4.3864e-02,  3.6321e-02,  9.6646e-02,
       -1.3829e-01,  3.8637e-01,  7.6962e-02, -1.1306e-01, -1.6083e+00,
        2.0062e-02, -5.2665e-02, -1.6597e-01,  1.2171e-01,  2.8945e-01,
       -1.7289e-01,  5.3035e-02, -2.7842e-01,  8.2376e-02, -1.1980e-02,
        3.7228e-02,  2.1867e-01,  1.5267e-01, -8.4361e-02, -3.1292e-01,
       -3.2093e-02,  2.0281e-01, -3.5910e-01,  1.6873e-02, -2.2996e-01,
        2.6044e-02,  3.5910e-01, -3.2431e-01, -5.4194e-01, -9.7742e-01,
        3.9198e-02, -1.7794e-01,  7.4200e-02, -4.1251e-02, -7.8917e-02,
        2.0646e-01, -6.6538e-02,  6.7401e-02,  1.4965e-01,  5.9107e-02,
       -3.7585e-02, -3.4672e-02,  5.5291e-02, -8.5636e-02,  9.1743e-02,
        4.9125e-01,  7.5606e-03, -3.0860e-01,  5.8902e-04, -8.6975e-02,
        3.9904e-01, -1.2695e-01,  2.2471e-01,  2.3658e-01,  3.0489e-01,
       -6.7363e-02,  3.5839e-01,  4.9703e-01,  4.1895e-01, -3.8494e-01,
       -2.6257e-01,  1.6049e-01, -1.0992e-01,  2.7477e-02,  1.49

In [47]:
(embedding_matrix[2363] == embedding_dict.get("reason")).all()

False

In [48]:
print(train_padded.shape)
print(train_labels.shape)

(97324, 600)
(97324, 9)


In [49]:
print(validation_padded.shape)
print(validation_labels.shape)

(24331, 600)
(24331, 9)


## 7. Build CNN Model

In [50]:
num_permissions = 9 #11
drop = 0.2

batch_size = 32
max_train_epochs = 300
validation_split = 6

early_stopping_patience = 16
early_stopping_delta = 0.02  # 2%

max_description_embeddings = 600
embedding_dim = 300  # +1 for flag

#downloaded_embedding_file = data_folder + "/word_embeddings/glove.6B.300d.txt"


conv_filters_num = 1024
conv_filters_sizes = [1, 2, 3]
dense_layers = [5000, 2500]
dropout = 0.2

heatmap_threshold = 0.49

In [51]:
# Function for class weights

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)

    print(class_series)
    print(class_labels)

    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(n_samples)
    print(n_classes)

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))
    #return class_weights

In [52]:
class_series = np.array(train_labels)
class_wt = generate_class_weights(class_series, multi_class=False, one_hot_encoded=True)
print(class_wt)
#class_wt = torch.tensor(class_wt)
#print(class_wt)

97324
9
{0: 0.661919433052444, 1: 0.6634219495569189, 2: 1.2259129098489716, 3: 1.0322430104789784, 4: 0.22413832810549636, 5: 0.7171415729012387, 6: 34.770989639156845, 7: 53.799889441680484, 8: 6.09226917057903}


In [53]:
def model_multiconv_1d(num_permissions):
    #embedding_dim = embedding_dim
    sequence_length = max_description_embeddings

    input_layer = Input(shape=(None,))

    conv_layers = []
    for filter_size in conv_filters_sizes:
        conv_layer_i = Embedding(num_words,
                                 output_dim=embedding_dim,
                                 input_length=sequence_length,
                                 weights=[embedding_matrix],
                                 trainable=False)(input_layer)
        conv_layer_i = Conv1D(filters=conv_filters_num,
                              kernel_size=filter_size,
                              padding='same',
                              activation='relu')(conv_layer_i)
        conv_layer_i = GlobalMaxPooling1D()(conv_layer_i)

        conv_layers.append(conv_layer_i)

    if len(conv_layers) == 1:
        previous_layer = conv_layers[0]
    else:
        concatenated_layer = concatenate(conv_layers, axis=-1)
        previous_layer = concatenated_layer

    for n_neurons in dense_layers:
        previous_layer = Dense(n_neurons, activation='relu')(previous_layer)
        previous_layer = Dropout(dropout)(previous_layer)

    output_layer = Dense(num_permissions, activation='sigmoid')(previous_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [54]:
model = model_multiconv_1d(num_permissions)

In [55]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])
              
#metrics=[metrics.fb_micro, metrics.fb_macro, metrics.precision, metrics.recall])
train_metric = 'val_fb_macro'

In [56]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    58574100    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    58574100    ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    58574100    ['input_1[0][0]']                
                                                                                              

In [57]:
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(train_padded, train_labels, 
          batch_size=batch_size, epochs=10, class_weight=class_wt,
          verbose=1, callbacks=[checkpoint], validation_data=(validation_padded, validation_labels))

Epoch 1/10
3042/3042 [==============================] - 569s 182ms/step - loss: 0.1595 - accuracy: 0.2682 - val_loss: 0.2360 - val_accuracy: 0.2844
Epoch 2/10
3042/3042 [==============================] - 553s 182ms/step - loss: 0.1310 - accuracy: 0.2934 - val_loss: 0.2176 - val_accuracy: 0.2858
Epoch 3/10
3042/3042 [==============================] - 590s 194ms/step - loss: 0.1110 - accuracy: 0.2970 - val_loss: 0.2151 - val_accuracy: 0.2979
Epoch 4/10
3042/3042 [==============================] - 552s 181ms/step - loss: 0.0931 - accuracy: 0.3017 - val_loss: 0.2196 - val_accuracy: 0.2953
Epoch 5/10
3042/3042 [==============================] - 590s 194ms/step - loss: 0.0832 - accuracy: 0.3052 - val_loss: 0.2386 - val_accuracy: 0.2842
Epoch 6/10
3042/3042 [==============================] - 589s 194ms/step - loss: 0.0720 - accuracy: 0.3107 - val_loss: 0.2318 - val_accuracy: 0.2884
Epoch 7/10
3042/3042 [==============================] - 547s 180ms/step - loss: 0.0685 - accuracy: 0.3139 - val_

## 8. Performance on Test Set

In [58]:
test_labels = df_test[target_list]
test_input = df_test["Clean_Description"]

In [59]:
test_sequences = tokenizer.texts_to_sequences(test_input)
test_padded = pad_sequences(
    test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [60]:
print(df_test.Clean_Description[0])
print(test_sequences[0])

poetry lover find best poetry poets urdu problem solved app app contains time best urdu poetry famous urdu poets offline urdu poetry collection best urdu poetry famous urdu poets like mohsin naqvi mirza ghalib saghir siddiqui others also contains romantic urdu poetry sad urdu poetry urdu ghazals offline urdu shayari urdu shayari app contains best ever poetry famous poets times app contains best variety urdu poetry categories romantic poetry sad poetry attitude poetry ghazals inspirational poetry main features offline poetry collection gallery view poetry collection zooming feature save share poetry images app contains offline poetry urdu ghazals urdu poetry poetry famous poets urdu shayari much more
[2815, 1981, 28, 15, 2815, 9152, 442, 690, 2422, 1, 1, 195, 13, 15, 442, 2815, 1000, 442, 9152, 142, 442, 2815, 124, 15, 442, 2815, 1000, 442, 9152, 17, 26288, 37084, 14260, 14674, 30509, 335, 14, 195, 589, 442, 2815, 1667, 442, 2815, 442, 9855, 142, 442, 697, 442, 697, 1, 195, 15, 494, 281

In [61]:
print(test_labels)

      Camera  Location  Microphone  Contacts  Storage  Phone  SMS  Call_Log  \
0          0         0           0         0        1      0    0         0   
1          0         0           0         0        1      0    0         0   
2          0         0           0         0        1      0    0         0   
3          0         0           0         0        0      0    0         0   
4          1         0           0         0        0      1    0         0   
...      ...       ...         ...       ...      ...    ...  ...       ...   
4619       0         1           0         0        0      0    0         0   
4620       0         0           0         0        0      0    0         0   
4621       0         0           0         0        1      0    0         0   
4622       0         1           1         0        1      0    0         0   
4623       0         0           0         0        0      0    0         0   

      Calendar  
0            0  
1            0  


In [62]:
print(test_padded.shape)
print(test_labels.shape)

(4624, 600)
(4624, 9)


In [63]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_padded)
print("predictions shape:", predictions.shape)

predictions shape: (4624, 9)


In [64]:
predictions[0:10]

array([[3.78265569e-04, 2.32471130e-03, 1.94393033e-05, 6.80311350e-05,
        3.91769633e-02, 2.59107794e-03, 6.14969224e-07, 4.30538583e-09,
        1.28920310e-05],
       [1.80578900e-06, 1.55568887e-05, 3.89848424e-07, 3.15264992e-06,
        4.21113241e-03, 4.01363141e-05, 1.30931443e-09, 8.02849757e-13,
        2.78678005e-08],
       [2.17507441e-05, 4.96457642e-06, 4.45515579e-05, 1.47671934e-04,
        4.21169261e-03, 8.02360300e-04, 2.52257337e-07, 2.68068878e-09,
        7.13674717e-06],
       [8.66502205e-07, 3.79620120e-04, 5.45803971e-08, 6.42189989e-03,
        1.03893345e-02, 7.61143342e-02, 1.04823121e-05, 1.21492036e-07,
        3.15775378e-06],
       [9.53132403e-05, 7.32674380e-04, 2.97838938e-06, 3.63712898e-03,
        1.65278129e-02, 6.04360737e-02, 3.14224017e-05, 5.98584109e-07,
        6.14649525e-06],
       [6.27905428e-01, 8.67290981e-03, 2.29924591e-03, 1.67271588e-03,
        9.64481771e-01, 7.96158686e-02, 2.86827481e-06, 1.27318060e-07,
        3.1

In [65]:
true_labels = test_labels.to_numpy()
true_labels[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0]])

In [66]:
np.save("predictions.npy", predictions)
loaded_predictions = np.load("predictions.npy")
print(loaded_predictions)

[[3.78265569e-04 2.32471130e-03 1.94393033e-05 ... 6.14969224e-07
  4.30538583e-09 1.28920310e-05]
 [1.80578900e-06 1.55568887e-05 3.89848424e-07 ... 1.30931443e-09
  8.02849757e-13 2.78678005e-08]
 [2.17507441e-05 4.96457642e-06 4.45515579e-05 ... 2.52257337e-07
  2.68068878e-09 7.13674717e-06]
 ...
 [3.58468533e-01 4.86980528e-01 1.86009146e-03 ... 4.05297091e-04
  6.61759696e-05 2.87043373e-03]
 [6.40523271e-04 9.54265714e-01 9.99431074e-01 ... 5.35393466e-11
  8.14271567e-12 1.78427072e-04]
 [1.09103075e-05 4.00412409e-06 2.41250238e-08 ... 3.04824141e-07
  2.37315945e-10 4.18350879e-07]]


In [67]:
np.save("true_labels.npy", true_labels)
loaded_true_labels = np.load("true_labels.npy")
print(loaded_true_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 9. Threshold Calculation

In [68]:
# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [69]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [70]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.0386 with F-Score: 0.6701
permission  1
Length of sequence: 10000
Best Threshold: 0.0805 with F-Score: 0.5552
permission  2
Length of sequence: 10000
Best Threshold: 0.1286 with F-Score: 0.5646
permission  3
Length of sequence: 10000
Best Threshold: 0.029 with F-Score: 0.5346
permission  4
Length of sequence: 10000
Best Threshold: 0.029 with F-Score: 0.7927
permission  5
Length of sequence: 10000
Best Threshold: 0.0844 with F-Score: 0.5145
permission  6
Length of sequence: 10000
Best Threshold: 0.0689 with F-Score: 0.1026
permission  7
Length of sequence: 10000
Best Threshold: 0.0892 with F-Score: 0.4211
permission  8
Length of sequence: 10000
Best Threshold: 0.175 with F-Score: 0.3929
-------------------------------------
optimal threshold tuning for f-score
[0.0386, 0.0805, 0.1286, 0.029, 0.029, 0.0844, 0.0689, 0.0892, 0.175]


In [71]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.004699999932199717 with G-Mean: 0.828
FPR: 0.1487, TPR: 0.8054
permission  1
Best Threshold: 0.004999999888241291 with G-Mean: 0.7565
FPR: 0.2091, TPR: 0.7235
permission  2
Best Threshold: 0.00019999999494757503 with G-Mean: 0.8149
FPR: 0.2519, TPR: 0.8876
permission  3
Best Threshold: 0.0017999999690800905 with G-Mean: 0.7971
FPR: 0.2289, TPR: 0.824
permission  4
Best Threshold: 0.08529999852180481 with G-Mean: 0.7761
FPR: 0.1845, TPR: 0.7386
permission  5
Best Threshold: 0.01600000075995922 with G-Mean: 0.762
FPR: 0.2563, TPR: 0.7807
permission  6
Best Threshold: 0.0 with G-Mean: 0.7736
FPR: 0.2686, TPR: 0.8182
permission  7
Best Threshold: 0.0 with G-Mean: 0.9418
FPR: 0.113, TPR: 1.0
permission  8
Best Threshold: 0.0006000000284984708 with G-Mean: 0.8209
FPR: 0.1804, TPR: 0.8222
-------------------------------------
ROC curve with G-mean threshold tuning
[0.0047, 0.005, 0.0002, 0.0018, 0.0853, 0.016, 0.0, 0.0, 0.0006]


## 10. Performance Score

In [ ]:
#Fscore micro for different thresholds-

In [72]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [73]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f_at_1(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

   
    acc[0] = f1_score(labels[:, 0], preds_th[:, 0])
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1])
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2])
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3])
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4])
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5])
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6])
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7])
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8])

   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [74]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("F1_CV2_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.6701
  Location  : 0.5552
  Microphone: 0.5646
  Contacts  : 0.5346
  Storage   : 0.7927
  Phone     : 0.5145
  SMS       : 0.1026
  Call_Log  : 0.4211
  Calendar  : 0.3929

  Average F1 score: 0.5053


In [ ]:
#Fscore micro for different thresholds-

In [75]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [76]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f1micro_accuracy(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

    acc[0] = f1_score(labels[:, 0], preds_th[:, 0], average='micro')
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1], average='micro')
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2], average='micro')
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3], average='micro')
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4], average='micro')
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5], average='micro')
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6], average='micro')
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7], average='micro')
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8], average='micro')
    
   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [77]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("F1Mic_CV2_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8904
  Location  : 0.8666
  Microphone: 0.9330
  Contacts  : 0.8821
  Storage   : 0.7727
  Phone     : 0.8367
  SMS       : 0.9924
  Call_Log  : 0.9976
  Calendar  : 0.9779

  Average F1 (micro) score: 0.9055


In [ ]:
#roc-auc score for different thresholds-

In [78]:
import numpy as np
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [79]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import roc_auc_score

def roc_auc(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_roc[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_roc[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_roc[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_roc[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_roc[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_roc[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_roc[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_roc[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_roc[8]


    acc[0] = roc_auc_score(labels[:, 0], preds_th[:, 0])
    acc[1] = roc_auc_score(labels[:, 1], preds_th[:, 1])
    acc[2] = roc_auc_score(labels[:, 2], preds_th[:, 2])
    acc[3] = roc_auc_score(labels[:, 3], preds_th[:, 3])
    acc[4] = roc_auc_score(labels[:, 4], preds_th[:, 4])
    acc[5] = roc_auc_score(labels[:, 5], preds_th[:, 5])
    acc[6] = roc_auc_score(labels[:, 6], preds_th[:, 6])
    acc[7] = roc_auc_score(labels[:, 7], preds_th[:, 7])
    acc[8] = roc_auc_score(labels[:, 8], preds_th[:, 8])
   

    #print(acc)
    return acc

In [80]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ROC_CV2_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.8281
  Location  : 0.7565
  Microphone: 0.8154
  Contacts  : 0.7973
  Storage   : 0.7768
  Phone     : 0.7614
  SMS       : 0.5000
  Call_Log  : 0.5000
  Calendar  : 0.8165

  Average ROC_AUC score: 0.7280


# ACNET performance

#### a) Data Preparation

In [81]:
## drive path
acnet_path = "/content/drive/MyDrive/MetadataCSV/acnet_dataset_preprocess.csv"

In [82]:
df_acnet = pd.read_csv(acnet_path) 
print(df_acnet.shape)

(1417, 14)


In [83]:
df_acnet = df_acnet.dropna(subset=['Clean_Description'])

In [84]:
df_acnet.head()

,app_id,description,Storage,Contacts,Location,Camera,Microphone,SMS,Call_Log,Phone,Calendar,Settings,Tasks,Clean_Description
0,0,ROOT is REQUIRED for automatic synchronization...,1,0,1,0,0,0,0,0,0,0,0,root is required for automatic synchronization...
1,1,This app delivers short scriptures containing ...,0,0,0,0,0,1,0,0,0,0,0,this app delivers short scriptures containing ...
2,2,This game is surprisingly simple and very addi...,0,0,0,0,0,0,0,0,0,0,0,this game is surprisingly simple and very addi...
3,3,It is an online RPG game based on LBS location...,0,0,1,0,0,0,0,0,0,0,0,it is an online rpg game based on lbs location...
4,4,Christmas is in the air. Get yourself in the h...,0,1,0,0,0,0,0,0,0,1,1,christmas is in the air. get yourself in the h...


In [85]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(remove_stopwords)

In [86]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(lambda x: remove_punct(x))

In [87]:
acnet_labels = df_acnet[target_list]
acnet_input = df_acnet["Clean_Description"]

In [88]:
acnet_sequences = tokenizer.texts_to_sequences(acnet_input)
acnet_padded = pad_sequences(
    acnet_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [89]:
print(acnet_padded.shape)
print(acnet_labels.shape)

(1414, 600)
(1414, 9)


#### b) Get Predictions

In [90]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(acnet_padded)
print("predictions shape:", predictions.shape)

predictions shape: (1414, 9)


In [91]:
acnet_labels = acnet_labels.to_numpy()

In [92]:
np.save("acnet_predictions.npy", predictions)
loaded_predictions = np.load("acnet_predictions.npy")

np.save("acnet_labels.npy", acnet_labels)
loaded_true_labels = np.load("acnet_labels.npy")

#### c) Threshold Calculation

In [93]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [94]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.019 with F-Score: 0.4704
permission  1
Length of sequence: 10000
Best Threshold: 0.039 with F-Score: 0.5559
permission  2
Length of sequence: 10000
Best Threshold: 0.0432 with F-Score: 0.5046
permission  3
Length of sequence: 10000
Best Threshold: 0.0228 with F-Score: 0.6022
permission  4
Length of sequence: 10000
Best Threshold: 0.014 with F-Score: 0.5967
permission  5
Length of sequence: 10000
Best Threshold: 0.7051 with F-Score: 0.4505
permission  6
Length of sequence: 10000
Best Threshold: 0.0011 with F-Score: 0.4793
permission  7
Length of sequence: 10000
Best Threshold: 0.0016 with F-Score: 0.3401
permission  8
Length of sequence: 10000
Best Threshold: 0.0253 with F-Score: 0.465
-------------------------------------
optimal threshold tuning for f-score
[0.019, 0.039, 0.0432, 0.0228, 0.014, 0.7051, 0.0011, 0.0016, 0.0253]


In [95]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.01209999993443489 with G-Mean: 0.7286
FPR: 0.2633, TPR: 0.7205
permission  1
Best Threshold: 0.02539999969303608 with G-Mean: 0.735
FPR: 0.2432, TPR: 0.7138
permission  2
Best Threshold: 0.0032999999821186066 with G-Mean: 0.7797
FPR: 0.1967, TPR: 0.7568
permission  3
Best Threshold: 0.023399999365210533 with G-Mean: 0.7082
FPR: 0.2125, TPR: 0.6368
permission  4
Best Threshold: 0.05990000069141388 with G-Mean: 0.5884
FPR: 0.4743, TPR: 0.6586
permission  5
Best Threshold: 0.17110000550746918 with G-Mean: 0.7713
FPR: 0.2292, TPR: 0.7719
permission  6
Best Threshold: 0.00019999999494757503 with G-Mean: 0.749
FPR: 0.2962, TPR: 0.7972
permission  7
Best Threshold: 0.0 with G-Mean: 0.7526
FPR: 0.3369, TPR: 0.8542
permission  8
Best Threshold: 0.008999999612569809 with G-Mean: 0.8242
FPR: 0.1489, TPR: 0.7981
-------------------------------------
ROC curve with G-mean threshold tuning
[0.0121, 0.0254, 0.0033, 0.0234, 0.0599, 0.1711, 0.0002, 0.0, 0.009]


#### d) Accuracy Score

In [ ]:
#Fscore micro for different thresholds-

In [96]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [97]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("ACNET_F1_CV2_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.4704
  Location  : 0.5559
  Microphone: 0.5046
  Contacts  : 0.6022
  Storage   : 0.5967
  Phone     : 0.4505
  SMS       : 0.4793
  Call_Log  : 0.3401
  Calendar  : 0.4650

  Average F1 score: 0.4961


In [ ]:
#Fscore micro for different thresholds-

In [98]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [99]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("ACNET_F1Mic_CV2_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.7532
  Location  : 0.7694
  Microphone: 0.8861
  Contacts  : 0.7412
  Storage   : 0.5297
  Phone     : 0.9137
  SMS       : 0.7956
  Call_Log  : 0.8628
  Calendar  : 0.8812

  Average F1 (micro) score: 0.7926


In [ ]:
#roc-auc score for different thresholds-

In [100]:
import numpy as np
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [101]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ACNET_ROC_CV2_N121k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.7286
  Location  : 0.7353
  Microphone: 0.7771
  Contacts  : 0.7110
  Storage   : 0.5913
  Phone     : 0.7670
  SMS       : 0.7422
  Call_Log  : 0.5000
  Calendar  : 0.8242

  Average ROC_AUC score: 0.7085
